In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import block_diag
from numpy.linalg import multi_dot
from scipy.linalg import sqrtm
from scipy.sparse import csgraph
from scipy import linalg
import os

In [2]:
N_TRIAL = 1000
N_ARMS = 100 #N_ARMS -> number of clients
N_FEATURE = 3
M = 29
K = 3

In [3]:
# X --> TBD now using x_{it} = [1,t,t^2] for all client i
Y_1 = np.genfromtxt('Y(noise0.1).csv',delimiter=',')
W = np.genfromtxt('W(noise0.1).csv',delimiter=',')
Beta = np.genfromtxt('Beta(noise0.1).csv',delimiter=',')
q = np.genfromtxt('q_init4.csv',delimiter=',')
c = np.genfromtxt('c_init4.csv',delimiter=',')

In [4]:
def make_regret(payoff, oracle):
    return np.cumsum(oracle - payoff)

def plot_regrets(results, oracle):
    [plt.plot(make_regret(payoff=x['r_payoff'], oracle=oracle), label="alpha: "+str(alpha)) for (alpha, x) in results.items()]

In [5]:
# X transformation from a sparse matrix
def X_reshape(X, X_tr, t, K, n_arms, n_feature):  #
  for arm in range(1, n_arms):
    X_tr = np.concatenate((X_tr,np.kron(np.identity(n = K),X[arm].reshape(-1,1))), axis = 1)
  return X_tr

# convert to a sparse matrix -> convert to a long sparse vector with flatten()
def X_to_X_m(X, t, arm_choice, n_arms, n_feature): 
  X_m = np.copy(X[t])
  for arm in np.arange(n_arms): # N x p
    if arm not in arm_choice:
      X_m[arm] = np.zeros(shape=n_feature)
  return X_m

In [6]:
# Create the F matrix
def constructAdjMatrix(W, n, threshold): #m
    Adj_mat = np.zeros(shape = (n, n))
    for ui in range(n):
        for uj in range(n):
            Adj_mat[ui][uj] = W[ui][uj]
        # trim the graph
            for i in range(n):
                if W[ui][i] <= threshold:
                    Adj_mat[ui][i] = 0;
#         Adj_mat[ui] /= sum(Adj_mat[ui])
    return Adj_mat
def constructLaplacianMatrix(W, n, Gepsilon):
    G = W.copy()
    #Convert adjacency matrix of weighted graph to adjacency matrix of unweighted graph
    for i in range(n):
        for j in range(n):
            if G[i][j] > 0:
                G[i][j] = 1
    L = csgraph.laplacian(G, normed = False)
    I = np.identity(n = G.shape[0])
    GW = I + Gepsilon*L  # W is a double stochastic matrix
    return GW.T

In [7]:
# Create the F matrix: (lda = 0 for CL-UCB wo)
lda = 0.01
threshold = 0.5
test_adj = constructAdjMatrix(W, N_ARMS, threshold)
test_F = constructLaplacianMatrix(test_adj, N_ARMS, lda)
FInv_Init = sqrtm(np.linalg.inv(np.kron(test_F, np.identity(n=K))))

In [8]:
#upload/download trigger
#UPLOAD
def upload(gammaU, IDclient, A_loc, A_up_buff):
    numerator = linalg.det(A_loc[IDclient] + A_up_buff[IDclient]) 
    denominator = linalg.det(A_loc[IDclient])
    if denominator == 0:
        return True
    else:
        check = numerator/denominator
        return check > gammaU
    # return numerator/denominator > gammaU

#DOWNLOAD
def download(gammaD, IDclient, A_gob, A_down_buff):
    numerator = linalg.det(A_gob + A_down_buff[IDclient])
    denominator = linalg.det(A_gob)
    #print(numerator/denominator)
    if denominator == 0:
        return True
    else:
        check = numerator/denominator
        return check > gammaD
    # return numerator/denominator > gammaD

def event_trigger(totalCommCost, IDclient, gammaU, gammaD, A_loc, A_up_buff, A_gob, A_down_buff, b_loc, b_up_buff, b_gob, b_down_buff, D_loc, D_up_buff, D_gob, \
                          D_down_buff, d_loc, d_up_buff, d_gob, d_down_buff, n_clients, n_feature, K):
    # check upload triggering event for A
    if upload(gammaU, IDclient, A_loc, A_up_buff):
        totalCommCost += 1
        # update server's statistics
        A_gob += A_up_buff[IDclient]
        b_gob += b_up_buff[IDclient]
                
        # update server's download buffer for other clients
        for clientID in np.arange(n_clients):
            if clientID != IDclient:
                A_down_buff[clientID] += A_up_buff[IDclient]
                b_down_buff[clientID] += b_up_buff[IDclient]
                        
        # clear client's upload buffer
        A_up_buff[IDclient] = np.zeros((K * n_feature, K * n_feature))
        b_up_buff[IDclient] = np.zeros(shape=K * n_feature)
        # print('up here A', totalCommCost)
        # print(A_up_buff[IDclient])
        # print((linalg.det(A_gob))/(linalg.det(A_gob-A_up_buff[IDclient])))
        # print(linalg.det(A_gob))
        # print(linalg.det(A_gob-A_down_buff[IDclient]))
        # check download triggering event for all clients
        for cli in np.arange(n_clients):
            if download(gammaD, cli, A_gob, A_down_buff):
                # print('here down')
                totalCommCost += 1
                
                # update client's local statistics, and clear server's download buffer
                A_loc[cli] += A_down_buff[cli]
                b_loc[cli] += b_down_buff[cli]
                
                # clear cserver's download buffer
                A_down_buff[cli] = np.zeros((K * n_feature, K * n_feature))
                b_down_buff[cli] = np.zeros(shape=K * n_feature)
        # print('up + down cost A', totalCommCost)       
    # check upload triggering event for D
    if upload(gammaU, IDclient, D_loc, D_up_buff):
        totalCommCost += 1
        # update server's statistics
        D_gob += D_up_buff[IDclient]
        d_gob += d_up_buff[IDclient]
                
        # update server's download buffer for other clients
        for clientID in np.arange(n_clients):
            if clientID != IDclient:
                D_down_buff[clientID] += D_up_buff[IDclient]
                d_down_buff[clientID] += d_up_buff[IDclient]
                        
        # clear client's upload buffer
        D_up_buff[IDclient] = np.zeros((n_clients * K, n_clients * K))
        d_up_buff[IDclient] = np.zeros(shape=n_clients * K)
        # print('up here D', totalCommCost)
        # check download triggering event for all clients
        for cli in np.arange(n_clients):
            if download(gammaD, cli, D_gob, D_down_buff):
                totalCommCost += 1
                
                # update client's local statistics, and clear server's download buffer
                D_loc[cli] += D_down_buff[cli]
                d_loc[cli] += d_down_buff[cli]
                
                # clear cserver's download buffer
                D_down_buff[cli] = np.zeros((n_clients * K, n_clients * K))
                d_down_buff[cli] = np.zeros(shape=n_clients * K)
        # print('up + down cost A D', totalCommCost) 
    return totalCommCost, A_loc, A_up_buff, A_gob, A_down_buff, b_loc, b_up_buff, b_gob, b_down_buff, D_loc, D_up_buff, D_gob, D_down_buff, d_loc, d_up_buff, d_gob, d_down_buff

In [9]:
def Fed_CLUCB(eta_1, eta_2, alpha_q, alpha_c, X, Y, init_q, init_c, m, K, FInv, X_to_X_m, X_reshape, oracle, gammaU, gammaD):
    n_trial, n_clients, n_feature = X.shape

    # 1.1. Output objects
    totalCommCost = 0
    client_choice = np.empty(shape=(n_trial, m), dtype=int)
    r_payoff = np.empty(n_trial)   
    c_payoff = np.empty(n_trial) 
    cum_regret = np.empty(n_trial)
    p = np.empty(shape=(n_trial, n_clients))
    cum_totalCommCost = np.empty(n_trial)
    
    # te_q = np.empty(shape = (n_trial + 1, K * n_feature)) #Kp x 1
    # te_C_tilde = np.empty(shape = (n_trial + 1, n_arms * K)) #NK x 1
    
    # 1.2. Intialize local statistics
    A_loc = np.array([eta_1 * np.identity(n=K * n_feature) for _ in np.arange(n_clients)])
    A_up_buff = np.array([np.zeros((K * n_feature, K * n_feature)) for _ in np.arange(n_clients)])
    b_loc = np.array([np.zeros(shape=K * n_feature)  for _ in np.arange(n_clients)])
    b_up_buff = np.array([np.zeros(shape=K * n_feature)  for _ in np.arange(n_clients)])
    q_loc = np.empty(shape = (n_trial + 1, n_clients, K * n_feature)) #Kp x 1
    
    D_loc = np.array([eta_2 * np.identity(n=n_clients * K) for _ in np.arange(n_clients)])
    D_up_buff = np.array([np.zeros((n_clients * K, n_clients * K)) for _ in np.arange(n_clients)])
    d_loc = np.array([np.zeros(shape=n_clients * K)  for _ in np.arange(n_clients)])
    d_up_buff = np.array([np.zeros(shape=n_clients * K)  for _ in np.arange(n_clients)])
    c_loc = np.empty(shape = (n_trial + 1, n_clients, n_clients * K)) #NK x 1
    
    #add initialization for each client
    for b in np.arange(n_clients): 
        q_loc[0, b] = init_q
        c_loc[0, b] = init_c
    
    # 1.3 Global statistics
    A_gob = eta_1 * np.identity(n=K * n_feature) 
    A_down_buff = np.array([np.zeros((K * n_feature, K * n_feature)) for _ in np.arange(n_clients)])  
    b_gob = np.zeros(shape=K * n_feature)
    b_down_buff = np.array([np.zeros(shape=K * n_feature)  for _ in np.arange(n_clients)])
    
    D_gob = eta_2 * np.identity(n=n_clients * K) 
    D_down_buff = np.array([np.zeros((n_clients * K, n_clients * K)) for _ in np.arange(n_clients)])  
    d_gob = np.zeros(shape=n_clients * K)
    d_down_buff = np.array([np.zeros(shape=n_clients * K)  for _ in np.arange(n_clients)])
    
    
    # 2. Algorithm
    for t in np.arange(n_trial):
        for a in np.arange(n_clients):
            #Calculate inv(A_loc[a]), inv(D_loc[a]), q_loc[t,a], c_loc[t,a]
            inv_A = np.linalg.inv(A_loc[a])
            inv_D = np.linalg.inv(D_loc[a])
            if t != 0:
                q_loc[t, a] = inv_A.dot(b_loc[a])
                c_loc[t, a] = inv_D.dot(d_loc[a])
            #X Transformation for q case 
            X_temp = X_to_X_m(X, t, [a], n_clients, n_feature)    
            X_tr_init = np.kron(np.identity(n = K),X_temp[0].reshape(-1,1))
            X_tr = X_reshape(X_temp, X_tr_init, t, K, n_clients, n_feature) #Kp x NK 
            
            #X Transformation for c case
            X_tilde = FInv.dot(X_to_X_m(X, t, [a], n_clients, n_feature).flatten()) #Np x 1
            
            #Calculate cb_q and cb_c
            #cb_q  
            X_q_a = X_tr.dot(FInv.dot(c_loc[t, a]))
            cb_q = alpha_q * np.sqrt(X_q_a.T.dot(inv_A).dot(X_q_a))
            
            #cb_c
            # q_block = (block_diag(*[q_loc[t, a].reshape((K, n_feature)) for _ in np.arange(n_clients)])).T #Np x NK
            q_block = np.kron(np.eye(n_clients,dtype=int),q_loc[t, a].reshape((K,n_feature)).T)
            X_c = q_block.T.dot(X_tilde)
            cb_c = alpha_c * np.sqrt((X_c).T.dot(inv_D).dot(X_c))
            
            #Predictions
            p[t, a] = (FInv.dot(c_loc[t, a]).T).dot(X_tr.T).dot(q_loc[t, a]) + cb_q + cb_c
            
        # The central server chooses m best clients
        # idx = np.argpartition(p[t], -m)[-m:]
        # chosen_clients = idx[np.argsort(-(p[t])[idx])]
        # for i in np.arange(m):
        #     client_choice[t][i] = chosen_clients[i]
            
        chosen_clients = p[t].argsort()[-m:][::-1]
        for i in np.arange(m):
            client_choice[t][i] = chosen_clients[i]

            # Update local statistics based on following conditions
        for chosen_client in client_choice[t]:
            # client local statistics update
            
            X_tr_chosen_temp = X_to_X_m(X, t, [chosen_client], n_clients, n_feature)
            X_tr_init_cs = np.kron(np.identity(n = K),X_tr_chosen_temp[0].reshape(-1,1)) 
            X_1_tr_chosen =  X_reshape(X_tr_chosen_temp, X_tr_init_cs, t, K, n_clients, n_feature)
            X_q = FInv.dot(c_loc[t, chosen_client]).dot(X_1_tr_chosen.T)
            
            X_tilde_chosen = FInv.dot(X_to_X_m(X, t, [chosen_client], n_clients, n_feature).flatten())
            # q_block_chosen = (block_diag(*[q_loc[t, chosen_client].reshape((K, n_feature)) for _ in np.arange(n_clients)])).T
            q_block_chosen = np.kron(np.eye(n_clients,dtype=int),q_loc[t, chosen_client].reshape((K,n_feature)).T)
            X_C_Tilde = q_block_chosen.T.dot(X_tilde_chosen)
            
            A_loc[chosen_client] = A_loc[chosen_client] + np.outer(X_q, X_q)
            A_up_buff[chosen_client] = A_up_buff[chosen_client] + np.outer(X_q, X_q)
            b_loc[chosen_client] = b_loc[chosen_client] + Y[t, chosen_client] * X_q
            b_up_buff[chosen_client] = b_up_buff[chosen_client] + Y[t, chosen_client] * X_q
            
            D_loc[chosen_client] = D_loc[chosen_client] + np.outer(X_C_Tilde, X_C_Tilde)
            D_up_buff[chosen_client] = D_up_buff[chosen_client] + np.outer(X_C_Tilde, X_C_Tilde)
            d_loc[chosen_client] = d_loc[chosen_client] + Y[t, chosen_client] * X_C_Tilde
            d_up_buff[chosen_client] = d_up_buff[chosen_client] + Y[t, chosen_client] * X_C_Tilde

            # check upload triggering event for each local statistics A, D
            totalCommCost, A_loc, A_up_buff, A_gob, A_down_buff, b_loc, b_up_buff, b_gob, b_down_buff, \
            D_loc, D_up_buff, D_gob, D_down_buff, d_loc, d_up_buff, d_gob, d_down_buff = \
            event_trigger(totalCommCost, chosen_client, gammaU, gammaD, A_loc, A_up_buff, A_gob, \
                          A_down_buff, b_loc, b_up_buff, b_gob, b_down_buff, D_loc, D_up_buff, D_gob, \
                          D_down_buff, d_loc, d_up_buff, d_gob, d_down_buff,n_clients, n_feature, K)
            
            #else: if do not pass the upload, then the statistics are still the same in local
               
        #else: for other clients not selected at round t, the statistics are still the same in local
        
                
        # Cumulative regret
        r_payoff[t] = np.sum([Y[t, choice] for choice in client_choice[t]])      
        cum_regret[t] = np.sum(oracle[0:t+1] - r_payoff[0:t+1])
        cum_totalCommCost[t] = totalCommCost
        if (t+1) % 50 == 0:
            print('TRIAL:',t,'DONE', '| cum_regret:', cum_regret[t])
            print('Total Communication cost:', totalCommCost)
        # print(cum_regret[t], totalCommCost)
        
    return dict(A_gob=A_gob, b_gob=b_gob, D_gob=D_gob, d_gob=d_gob, q_loc=q_loc, c_loc = c_loc, p = p, client_choice = client_choice, r_payoff = r_payoff, totalCommCost=totalCommCost, cum_totalCommCost=cum_totalCommCost)

In [10]:
# Create X_i = [1, t, t^2]
X_1_lst = []
for T in np.arange(N_TRIAL):
  X_1t_lst = []
  for arm in np.arange(N_ARMS):
    temp = []
    temp.append(1)
    temp.append(0.001*(T+1))
    temp.append((0.001*(T+1))**2)
    X_1t_lst.append(np.array(temp))
  X_1_lst.append(np.array(X_1t_lst))
X_1 = np.array(X_1_lst)

In [11]:
oracle_lst = []
true_choice = []
new_y = -1 * Y_1 + 30 #
for t in np.arange(N_TRIAL):
  # Find indices of M highest arms
  all_reward_t = [new_y.T[t, arm] for arm in np.arange(N_ARMS)]
  chosen_arms = np.array(all_reward_t).argsort()[-M:][::-1]
  # Sum of M highest rewards
  oracle_payoff_t = np.sum([new_y.T[t, choice] for choice in chosen_arms])
  # Append to the list
  oracle_lst.append(oracle_payoff_t)
  true_choice.append(chosen_arms)
oracle_case1 = np.array(oracle_lst)

In [12]:
# Initialize q and C
# vec_q: q (Kp x 1)
np.random.seed(3) #3 #59
vec_q = q[~np.isnan(q)]
# vec_C: C (NK x 1)
np.random.seed(42)
vec_C = c

In [13]:
alpha_to_test = [0.5]
print('M:', M, 'lda:', lda, 'T', threshold)
results_dict = {alpha: Fed_CLUCB(eta_1 = 0.3, eta_2 = 0.3, alpha_q = 1, alpha_c = alpha, X=X_1, Y=(-1 * Y_1 + 30).T, init_q=vec_q, init_c=vec_C,m=M, K = K, FInv=FInv_Init, X_to_X_m=X_to_X_m, X_reshape=X_reshape, oracle=oracle_case1, gammaU=1, gammaD=1)\
                for alpha in alpha_to_test}

M: 29 lda: 0.01 T 0.5


C:\Users\ARTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:75: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ARTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ARTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:102: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ARTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ARTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ARTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:106: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\ARTY\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: ComplexWarning: Casting complex values to real discards the imaginar

TRIAL: 49 DONE | cum_regret: 221.88452380690637
Total Communication cost: 290000
TRIAL: 99 DONE | cum_regret: 228.56053189740103
Total Communication cost: 580000
TRIAL: 149 DONE | cum_regret: 233.91926224030502
Total Communication cost: 870000
TRIAL: 199 DONE | cum_regret: 236.29461472749273
Total Communication cost: 1160000
TRIAL: 249 DONE | cum_regret: 239.23476320032952
Total Communication cost: 1450000
TRIAL: 299 DONE | cum_regret: 241.98634232865962
Total Communication cost: 1740000
TRIAL: 349 DONE | cum_regret: 245.1147768150916
Total Communication cost: 2030000
TRIAL: 399 DONE | cum_regret: 247.98984435565353
Total Communication cost: 2320000
TRIAL: 449 DONE | cum_regret: 249.48704287214963
Total Communication cost: 2610000
TRIAL: 499 DONE | cum_regret: 252.306723295925
Total Communication cost: 2900000
TRIAL: 549 DONE | cum_regret: 254.3752491058299
Total Communication cost: 3190000
TRIAL: 599 DONE | cum_regret: 256.57496620334007
Total Communication cost: 3480000
TRIAL: 649 DO

In [14]:
# reward data
# test_lst = [x['c_payoff'] for (alpha, x) in results_dict.items()]
# df = pd.DataFrame(test_lst[0]) #index 4 is for alpha = 10 
# df.to_csv('C2_COCOlda5_reward(new).csv', header=False)

In [21]:
test_lst = [x['cum_totalCommCost'] for (alpha, x) in results_dict.items()]
df = pd.DataFrame(test_lst[0]) #index 4 is for alpha = 10 
df.to_csv('fed_C1_CLUCB_CommCost_29(1000).csv', header=False)
# # regret data
test_lst = [make_regret(payoff=x['r_payoff'], oracle=oracle_case1) for (alpha, x) in results_dict.items()]
df = pd.DataFrame(test_lst[0])
df.to_csv('fed_C1_CLUCB_Regret_29(1000).csv', header=False)

In [16]:
#before fixing
# TRIAL: 999 DONE | cum_regret: 1438.6521529268216
# Total Communication cost: 5784500
# TRIAL: 1999 DONE | cum_regret: 2619.0977275582654
# Total Communication cost: 11584500
# TRIAL: 2999 DONE | cum_regret: 3824.3587813563045
# Total Communication cost: 17384500
# TRIAL: 3999 DONE | cum_regret: 5067.483481510004
# Total Communication cost: 23184500

In [17]:
# #Update A, b, D, d for each selected arm
#         te_q[t] = q[t]
#         te_C_tilde[t] = C_tilde[t]
#         for j in range(5):
#             for chosen_arm in arm_choice[t]:
#                 X_tr_chosen_temp = X_to_X_m(X, t, [chosen_arm], n_arms, n_feature)
#                 X_tr_init_cs = np.kron(np.identity(n = K),X_tr_chosen_temp[0].reshape(-1,1))
#                 X_1_tr_chosen =  X_reshape(X_tr_chosen_temp, X_tr_init_cs, t, K, n_arms, n_feature)
            
#                 #x_tr for c case
#                 X_tilde_chosen = FInv.dot(X_to_X_m(X, t, [chosen_arm], n_arms, n_feature).flatten())
#                 # q_block_chosen = (block_diag(*[q[t].reshape((K, n_feature)) for _ in np.arange(n_arms)])).T
#                 q_block_chosen = (block_diag(*[te_q[t].reshape((K, n_feature)) for _ in np.arange(n_arms)])).T
            
#               #Update  
#                 X_q = FInv.dot(te_C_tilde[t]).dot(X_1_tr_chosen.T)
#                 X_C_Tilde = q_block_chosen.T.dot(X_tilde_chosen)
#                 A = A + np.outer(X_q, X_q)
#                 b = b + Y[t, chosen_arm] * X_q
#                 D = D + np.outer(X_C_Tilde, X_C_Tilde)           
#                 d = d + Y[t, chosen_arm] * X_C_Tilde
        
#             #inverse calculation
#             inv_A = np.linalg.inv(A)
#             inv_D = np.linalg.inv(D)
#             # q[t + 1] = inv_A.dot(b)
#             # C_tilde[t + 1] = inv_D.dot(d)
#             te_q[t] = inv_A.dot(b)
#             te_C_tilde[t] = inv_D.dot(d)
#         q[t + 1] = te_q[t]
#         C_tilde[t + 1] = te_C_tilde[t]

In [18]:
# 55.15406097592552 200
# 56.5470668972724 6000
# 57.66166899435788 11800
# 58.57117181927006 17600
# 59.603830862880585 23400
# 60.66058469071936 29200
# 61.88709555192938 35000
# 62.81667745120245 40800
# 63.981179003855345 46600
# 65.01065124519715 52400
# 66.14501741812558 58200
# 67.61661288186545 64000
# 68.92037049361903 69800
# 70.34718278983871 75600
# 71.74840100753994 81400
# 72.97342725925085 87200
# 74.17288217513743 93000
# 75.72587658596743 98800
# 76.7031667319325 104600
# 77.88126692095628 110400
# 78.92312025887449 116200
# 80.19707505286024 122000
# 81.32968673440723 127800
# 82.43243615967793 133600
# 83.46928878962012 139400
# 84.53125414132089 145200
# 85.71414986097918 151000
# 86.48765802973895 156800
# 87.8443959063195 162600
# 88.92110996298348 168400
# 90.31589817129051 174200
# 91.43358242041751 180000
# 92.63389731474129 185800
# 93.79425865056487 191600
# 95.00305973737099 197400
# 95.89900834220012 203200
# 97.18838139392659 209000
# 98.15015812739907 214800
# 99.46967755289606 220600
# 100.66066204216295 226400
# 101.81658388862894 232200
# 103.09120982619035 238000
# 104.15524629784838 243800
# 105.10637592150039 249600
# 106.33608302428746 255400
# 107.62583182758894 261200
# 108.34105273321194 267000
# 109.58014512751805 272800
# 110.79078273151308 278600
# 112.02559792854407 284400
# 113.38176288365051 290200
# 114.548794501522 296000
# 115.9497740330155 301800
# 117.03689950618096 307600
# 117.99024660224723 313400
# 119.2394790035982 319200
# 119.89592791825658 325000
# 121.01606196991278 330800
# 122.21966283756325 336600
# 123.26858745561287 342400
# 124.55676316571694 348200
# 125.58469109581264 354000
# 126.7766387570976 359800
# 127.71465573176329 365600
# 128.90017459518265 371400
# 129.9241650069573 377200
# 131.0594988985507 383000
# 132.27200727802187 388800
# 133.39063159345312 394600
# 134.52292320556174 400400
# 135.50386630299133 406200
# 136.44327591433452 412000
# 137.48022155867446 417800
# 138.59991474421287 423600
# 139.56065110060769 429400
# 140.8740614312433 435200
# 142.30743021881923 441000
# 143.5443181398485 446800
# 144.78732106208201 452600
# 146.09531341809233 458400
# 147.02388857566513 464200
# 148.11670190943528 470000
# 149.5547523570488 475800
# 150.87021428875033 481600
# 151.655756267222 487400
# 152.7648374382646 493200
# 153.7217875256937 499000
# 154.80477738889292 504800
# 155.9739642221901 510600
# 157.24712815298918 516400
# 158.50112085413625 522200
# 159.90046080256857 528000
# 161.08558642253826 533800
# 162.30111879273696 539600
# 163.59253260420346 545400
# 164.72986063660824 551200
# 166.05095201742415 557000
# 167.1207549958462 562800
# 168.40539910853371 568600
# 169.3997655739087 574400

In [19]:
# for debugging 
#def Fed_CLUCB(eta_1, eta_2, alpha_q, alpha_c, X, Y, init_q, init_c, m, K, FInv, X_to_X_m, X_reshape, oracle, gammaU, gammaD):
#     n_trial, n_clients, n_feature = X.shape

#     # 1.1. Output objects
#     totalCommCost = 0
#     client_choice = np.empty(shape=(n_trial, m), dtype=int)
#     r_payoff = np.empty(n_trial)   
#     c_payoff = np.empty(n_trial) 
#     cum_regret = np.empty(n_trial)
#     p = np.empty(shape=(n_trial, n_clients))
    
#     # te_q = np.empty(shape = (n_trial + 1, K * n_feature)) #Kp x 1
#     # te_C_tilde = np.empty(shape = (n_trial + 1, n_arms * K)) #NK x 1
    
#     # 1.2. Intialize local statistics
#     A_loc = np.array([eta_1 * np.identity(n=K * n_feature) for _ in np.arange(n_clients)])
#     A_up_buff = np.array([np.zeros((K * n_feature, K * n_feature)) for _ in np.arange(n_clients)])
#     b_loc = np.array([np.zeros(shape=K * n_feature)  for _ in np.arange(n_clients)])
#     b_up_buff = np.array([np.zeros(shape=K * n_feature)  for _ in np.arange(n_clients)])
#     q_loc = np.empty(shape = (n_trial + 1, n_clients, K * n_feature)) #Kp x 1
    
#     D_loc = np.array([eta_2 * np.identity(n=n_clients * K) for _ in np.arange(n_clients)])
#     D_up_buff = np.array([np.zeros((n_clients * K, n_clients * K)) for _ in np.arange(n_clients)])
#     d_loc = np.array([np.zeros(shape=n_clients * K)  for _ in np.arange(n_clients)])
#     d_up_buff = np.array([np.zeros(shape=n_clients * K)  for _ in np.arange(n_clients)])
#     c_loc = np.empty(shape = (n_trial + 1, n_clients, n_clients * K)) #NK x 1
    
#     #add initialization for each client
#     for b in np.arange(n_clients): 
#         q_loc[0, b] = init_q
#         c_loc[0, b] = init_c
    
#     # 1.3 Global statistics
#     A_gob = eta_1 * np.identity(n=K * n_feature) 
#     A_down_buff = np.array([np.zeros((K * n_feature, K * n_feature)) for _ in np.arange(n_clients)])  
#     b_gob = np.zeros(shape=K * n_feature)
#     b_down_buff = np.array([np.zeros(shape=K * n_feature)  for _ in np.arange(n_clients)])
    
#     D_gob = eta_2 * np.identity(n=n_clients * K) 
#     D_down_buff = np.array([np.zeros((n_clients * K, n_clients * K)) for _ in np.arange(n_clients)])  
#     d_gob = np.zeros(shape=n_clients * K)
#     d_down_buff = np.array([np.zeros(shape=n_clients * K)  for _ in np.arange(n_clients)])
    
    
#     # 2. Algorithm
#     for t in np.arange(n_trial):
#         for a in np.arange(n_clients):
#             #Calculate inv(A_loc[a]), inv(D_loc[a]), q_loc[t,a], c_loc[t,a]
#             inv_A = np.linalg.inv(A_loc[a])
#             inv_D = np.linalg.inv(D_loc[a])
#             if t != 0:
#                 q_loc[t, a] = inv_A.dot(b_loc[a])
#                 c_loc[t, a] = inv_D.dot(d_loc[a])
#             #X Transformation for q case 
#             X_temp = X_to_X_m(X, t, [a], n_clients, n_feature)    
#             X_tr_init = np.kron(np.identity(n = K),X_temp[0].reshape(-1,1))
#             X_tr = X_reshape(X_temp, X_tr_init, t, K, n_clients, n_feature) #Kp x NK 
            
#             #X Transformation for c case
#             X_tilde = FInv.dot(X_to_X_m(X, t, [a], n_clients, n_feature).flatten()) #Np x 1
            
#             #Calculate cb_q and cb_c
#             #cb_q  
#             X_q_a = X_tr.dot(FInv.dot(c_loc[t, a]))
#             cb_q = alpha_q * np.sqrt(X_q_a.T.dot(inv_A).dot(X_q_a))
            
#             #cb_c
#             # q_block = (block_diag(*[q_loc[t, a].reshape((K, n_feature)) for _ in np.arange(n_clients)])).T #Np x NK
#             q_block = np.kron(np.eye(n_clients,dtype=int),q_loc[t, a].reshape((K,n_feature)).T)
#             X_c = q_block.T.dot(X_tilde)
#             cb_c = alpha_c * np.sqrt((X_c).T.dot(inv_D).dot(X_c))
            
#             #Predictions
#             p[t, a] = (FInv.dot(c_loc[t, a]).T).dot(X_tr.T).dot(q_loc[t, a]) + cb_q + cb_c
            
#         # The central server chooses m best clients
#         # idx = np.argpartition(p[t], -m)[-m:]
#         # chosen_clients = idx[np.argsort(-(p[t])[idx])]
#         # for i in np.arange(m):
#         #     client_choice[t][i] = chosen_clients[i]
            
#         chosen_clients = p[t].argsort()[-m:][::-1]
#         for i in np.arange(m):
#             client_choice[t][i] = chosen_clients[i]
        
#         print(client_choice[t])
#             # Update local statistics based on following conditions
#         for chosen_client in client_choice[t]:
#             print('Client No.:', chosen_client)
#             # client local statistics update
            
#             X_tr_chosen_temp = X_to_X_m(X, t, [chosen_client], n_clients, n_feature)
#             X_tr_init_cs = np.kron(np.identity(n = K),X_tr_chosen_temp[0].reshape(-1,1)) 
#             X_1_tr_chosen =  X_reshape(X_tr_chosen_temp, X_tr_init_cs, t, K, n_clients, n_feature)
#             X_q = FInv.dot(c_loc[t, chosen_client]).dot(X_1_tr_chosen.T)
#             print(c_loc[t, chosen_client])
#             print('c_loc[t, chosen_client]----------------------------------------')
#             # print(X_tr_chosen_temp[chosen_client])
#             # print('X_tr_chosen_temp[chosen_client]----------------------------------------')
#             # print(X_tr_chosen_temp)
#             # print('X_tr_chosen_temp----------------------------------------')
#             # print(X_tr_init_cs)
#             # print('X_tr_init_cs----------------------------------------')
#             # print(X_1_tr_chosen)
#             # print('X_1_tr_chosen----------------------------------------')
#             print(X_q.real)
#             print('X_q.real----------------------------------------')
            
#             X_tilde_chosen = FInv.dot(X_to_X_m(X, t, [chosen_client], n_clients, n_feature).flatten())
#             # q_block_chosen = (block_diag(*[q_loc[t, chosen_client].reshape((K, n_feature)) for _ in np.arange(n_clients)])).T
#             q_block_chosen = np.kron(np.eye(n_clients,dtype=int),q_loc[t, chosen_client].reshape((K,n_feature)).T)
#             X_C_Tilde = q_block_chosen.T.dot(X_tilde_chosen)
            
            
#             print(X_tilde_chosen.real)
#             print('X_tilde_chosen.real----------------------------------------')
#             print(q_block_chosen)
#             print('q_block_chosen----------------------------------------')
#             print(X_C_Tilde.real)
#             print('X_C_Tilde.real----------------------------------------')
            
#             A_loc[chosen_client] = A_loc[chosen_client] + np.outer(X_q, X_q)
#             A_up_buff[chosen_client] = A_up_buff[chosen_client] + np.outer(X_q, X_q)
#             b_loc[chosen_client] = b_loc[chosen_client] + Y[t, chosen_client] * X_q
#             b_up_buff[chosen_client] = b_up_buff[chosen_client] + Y[t, chosen_client] * X_q
            
#             D_loc[chosen_client] = D_loc[chosen_client] + np.outer(X_C_Tilde, X_C_Tilde)
#             D_up_buff[chosen_client] = D_up_buff[chosen_client] + np.outer(X_C_Tilde, X_C_Tilde)
#             d_loc[chosen_client] = d_loc[chosen_client] + Y[t, chosen_client] * X_C_Tilde
#             d_up_buff[chosen_client] = d_up_buff[chosen_client] + Y[t, chosen_client] * X_C_Tilde
            
#             print(A_loc[chosen_client])
#             print('A_loc[chosen_client]----------------------------------------')
#             print(np.outer(X_q, X_q).real)
#             print('np.outer(X_q, X_q).real----------------------------------------')
#             print(A_up_buff[chosen_client])
#             print('A_up_buff[chosen_client]----------------------------------------')
#             # check upload triggering event for each local statistics A, D
#             totalCommCost, A_loc, A_up_buff, A_gob, A_down_buff, b_loc, b_up_buff, b_gob, b_down_buff, \
#             D_loc, D_up_buff, D_gob, D_down_buff, d_loc, d_up_buff, d_gob, d_down_buff = \
#             event_trigger(totalCommCost, chosen_client, gammaU, gammaD, A_loc, A_up_buff, A_gob, \
#                           A_down_buff, b_loc, b_up_buff, b_gob, b_down_buff, D_loc, D_up_buff, D_gob, \
#                           D_down_buff, d_loc, d_up_buff, d_gob, d_down_buff,n_clients, n_feature, K)
            
#             #else: if do not pass the upload, then the statistics are still the same in local
               
#         #else: for other clients not selected at round t, the statistics are still the same in local
        
                
#         # Cumulative regret
#         r_payoff[t] = np.sum([Y[t, choice] for choice in client_choice[t]])      
#         cum_regret[t] = np.sum(oracle[0:t+1] - r_payoff[0:t+1])
#         # if (t+1) % 1000 == 0:
#         #     print('TRIAL:',t,'DONE', '| cum_regret:', cum_regret[t])
#         #     print('Total Communication cost:', totalCommCost)
#         print(cum_regret[t], totalCommCost)
        
#     return dict(A_gob=A_gob, b_gob=b_gob, D_gob=D_gob, d_gob=d_gob, q_loc=q_loc, c_loc = c_loc, p = p, client_choice = client_choice, r_payoff = r_payoff, totalCommCost=totalCommCost)